In [ ]:
import logging
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import numpy as np
import geopandas as gpd
from pathlib import Path

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s %(message)s",
)

In [ ]:
spatial_resolution = 10
shape_folder = Path(f"data/SHAPE")
shape_file = shape_folder / "shapefile.shp"
output_folder = Path(f"results/mask.tif")

In [ ]:
gdf = gpd.read_file(shape_file)
gdf = gdf.to_crs("EPSG:32718")

bounds = gdf.total_bounds

cols = int((bounds[2] - bounds[0]) / spatial_resolution)
rows = int((bounds[3] - bounds[1]) / spatial_resolution)

fill_value = 0

gdf['Tipo'] = gdf['Tipo'].str.lower()

# Exclude some values.
exclude = ['glaciar en roca']
gdf = gdf[~gdf['Tipo'].isin(exclude)]
gdf['TipoInt'] = gdf['Tipo'].astype('category').cat.codes
gdf['TipoInt'] = gdf['TipoInt'] + 1

logging.info(f"""
size: cols={cols}, rows={rows}
bounds: {bounds}
values: {gdf['Tipo'].unique()}
values in numeric: {gdf['TipoInt'].unique()}
fill_value: {fill_value}
""")

data = rasterize(
    [(geom, value) for geom, value in zip(gdf.geometry, gdf.TipoInt)],
    out_shape=(rows, cols),
    transform=from_origin(bounds[0], bounds[3], spatial_resolution, spatial_resolution),
    fill=fill_value,
    dtype=np.uint8,
    all_touched=True
)

with rasterio.open(
    output_folder,
    'w',
    driver='GTiff',
    height=rows,
    width=cols,
    count=1,
    dtype=np.uint8,
    crs="EPSG:32718",
    transform=from_origin(bounds[0], bounds[3], spatial_resolution, spatial_resolution),
) as dst:
    dst.write(data, 1)

logging.info(f"Mask saved in {output_folder}")